In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100000)

# Зареждане на данните

In [ ]:
dataset = pd.read_csv('../data/prepared-car-offers.csv')

In [ ]:
dataset.sample(20)

In [ ]:
dataset.dtypes

In [ ]:
dataset = dataset.drop(columns='Region')

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler, TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.metrics import PredictionErrorDisplay

In [ ]:
max_abs_scaling_num_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', MaxAbsScaler())
])

target_enc_preprocessor = Pipeline([
    ('target_encoder', TargetEncoder(target_type='continuous', smooth=0.2))
])

ohe_cat_preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

columns_for_target_encoding = dataset[dataset.select_dtypes(include=object).columns].nunique()[dataset.nunique() > 6].index
target_ohe_enc_cat_preprocessor = ColumnTransformer([
    ('target_encoding', target_enc_preprocessor, columns_for_target_encoding),
    ('one_hot_encoding', ohe_cat_preprocessor, make_column_selector(dtype_include=object))
], remainder='passthrough')

default_preprocessor = ColumnTransformer([
    ('categorical', target_ohe_enc_cat_preprocessor, make_column_selector(dtype_include=object)),
    ('numerical', max_abs_scaling_num_preprocessor, make_column_selector(dtype_include=np.number)),
], remainder='passthrough')

random_forest_reg_pipeline = Pipeline([
    ('preprocessor', default_preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1))
])

In [ ]:
y = dataset['Price']
X = dataset.drop(columns='Price')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = random_forest_reg_pipeline.predict(X_test)

print('Root Mean Squared Error: ', root_mean_squared_error(y_test, y_pred))
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))

# Редуциране на пространството на характеристиките

## Премахване на колинеарни характеристики

In [ ]:
corr_matrix = dataset.select_dtypes(include=np.number).corr()

plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix, cmap='coolwarm')

In [ ]:
excluded_columns = ['Extras_TV', 'Extras_Tiptronic', 'Extras_USB', 'Extras_IN\\AUX изводи', 'Extras_Климатик']

no_colinearity_df = dataset.copy()
no_colinearity_df = no_colinearity_df.loc[:, no_colinearity_df.columns.difference(excluded_columns)]

y = no_colinearity_df['Price']
X = no_colinearity_df.drop(columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = random_forest_reg_pipeline.predict(X_test)

print('Средноквадратична грешка (RMSE): ', root_mean_squared_error(y_test, y_pred))
print('Средна абсолютна грешка (MAE): ', mean_absolute_error(y_test, y_pred))

## Премахване на колона: `Make`, `Model`, `EuroStandard`, `Color`

In [ ]:
excluded_columns = ['Make', 'Model', 'EuroStandard', 'Color']

for column in excluded_columns:
    df_excluded_column = dataset.drop(columns=column)

    columns_for_target_encoding = \
    df_excluded_column[df_excluded_column.select_dtypes(include=object).columns].nunique()[
        df_excluded_column.nunique() > 6].index

    cat_preproccessor = ColumnTransformer([
        ('target_encoding', target_enc_preprocessor, columns_for_target_encoding),
        ('one_hot_encoding', ohe_cat_preprocessor, make_column_selector(dtype_include=object))
    ], remainder='passthrough')

    preprocessor = ColumnTransformer([
        ('categorical', cat_preproccessor, make_column_selector(dtype_include=object)),
        ('numerical', max_abs_scaling_num_preprocessor, make_column_selector(dtype_include=np.number)),
    ], remainder='passthrough')

    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
    ])

    y = df_excluded_column['Price']
    X = df_excluded_column.drop(columns=['Price'])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_pipeline.fit(X_train, y_train)
    y_pred = rf_pipeline.predict(X_test)

    print('-' * 50)
    print('Резултати с премахната колона: ' + column)
    print(f'Средноквадратична грешка (RMSE) без {column}: ', root_mean_squared_error(y_test, y_pred))
    print(f'Средна абсолютна грешка (MAE) без {column}: ', mean_absolute_error(y_test, y_pred))

## Премахване на характеристики с ниска важност

In [ ]:
from sklearn.feature_selection import SelectFromModel

most_important_features_pipeline = Pipeline([
    ('preprocessor', default_preprocessor),
    ('feature_selection', SelectFromModel(RandomForestRegressor(random_state=42, n_jobs=-1), threshold=0.01)),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

y = dataset['Price']
X = dataset.drop(columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

most_important_features_pipeline.fit(X_train, y_train)
y_pred = most_important_features_pipeline.predict(X_test)

print('Средноквадратична грешка (RMSE): ', root_mean_squared_error(y_test, y_pred))
print('Средна абсолютна грешка (MAE): ', mean_absolute_error(y_test, y_pred))

## Премахване на характеристики с ниска взаимна информация

In [ ]:
from sklearn.feature_selection import mutual_info_regression

y = dataset['Price']
X = dataset.drop(columns=['Price'])

numerical_features = X.select_dtypes(include=np.number)
X = X[numerical_features.columns]
X = X.fillna(0)

X = default_preprocessor.fit_transform(X)

mutual_info = mutual_info_regression(X, y)

mutual_info_df = pd.DataFrame(data=zip(numerical_features.columns, mutual_info),
                              columns=['Feature', 'Mutual Information'])

In [ ]:
mutual_info_df.sort_values(by='Mutual Information', ascending=False)

In [ ]:
selected_features = mutual_info_df[mutual_info_df['Mutual Information'] >= 0.0001]['Feature'].values

selected_features

In [ ]:
selected_num_features = dataset[selected_features]
cat_features = dataset.select_dtypes(include=object)

y = dataset['Price']
X = pd.concat([selected_num_features, cat_features], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = random_forest_reg_pipeline.predict(X_test)

print('Средноквадратична грешка (RMSE): ', root_mean_squared_error(y_test, y_pred))
print('Средна абсолютна грешка (MAE): ', mean_absolute_error(y_test, y_pred))

## Премахване на характеристики с ниска корелация с целевата променлива

In [ ]:
correlation_with_price = dataset[dataset.drop(columns='Price').select_dtypes(include=np.number).columns].corrwith(dataset['Price'])

correlation_with_price_df = pd.DataFrame(np.abs(correlation_with_price), columns=['Correlation with Price'])
correlation_with_price_df.sort_values(by='Correlation with Price', ascending=False)

In [ ]:
correlated_features = correlation_with_price_df[correlation_with_price_df['Correlation with Price'] >= 0.5].index
cat_features = dataset.select_dtypes(include=object).columns
selected_features = correlated_features.union(cat_features)
selected_features

In [ ]:
y = dataset['Price']
X = dataset[selected_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = random_forest_reg_pipeline.predict(X_test)

print('Средноквадратична грешка (RMSE): ', root_mean_squared_error(y_test, y_pred))
print('Средна абсолютна грешка (MAE): ', mean_absolute_error(y_test, y_pred))

In [ ]:
dataset = dataset[selected_features.append(pd.Index(['Price']))]
dataset

# Откриване на най-добрите хиперпараметри за модела

In [ ]:
from sklearn.model_selection import GridSearchCV

import os
os.environ['JOBLIB_TIMEOUT'] = '300'

y = dataset['Price']
X = dataset.drop(columns='Price')

param_grid = {
    'model__n_estimators': [50, 100, 150, 200],
    'model__max_depth': [None, 10, 20, 30, 50],
    'model__min_samples_split': [2, 5, 10, 25],
    'model__min_samples_leaf': [None, 1, 2, 4, 10, 20, 50],
    'model__max_leaf_nodes': [None, 10, 20, 50, 100]
}

grid_search = GridSearchCV(random_forest_reg_pipeline, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X, y)

print('Best parameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

best_random_forest_reg_pipeline = grid_search.best_estimator_

In [ ]:
y = dataset['Price']
X = dataset.drop(columns='Price')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = best_random_forest_reg_pipeline.predict(X_test)

print('Root Mean Squared Error: ', root_mean_squared_error(y_test, y_pred))
print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))

# Анализиране на грешките

In [ ]:
y = dataset['Price']
X = dataset.drop(columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_random_forest_reg_pipeline.fit(X_train, y_train)
y_pred = best_random_forest_reg_pipeline.predict(X_test)

print('Средноквадратична грешка (RMSE): ', root_mean_squared_error(y_test, y_pred))
print('Средна абсолютна грешка (MAE): ', mean_absolute_error(y_test, y_pred))

PredictionErrorDisplay.from_predictions(y_test, y_pred, kind='residual_vs_predicted',
                                        scatter_kwargs={'alpha': 0.5, 'marker': 'x'}, subsample=None)

In [ ]:
errors = pd.Series(np.abs(y_test - y_pred))

errors_df = X_test.copy()
errors_df['Error'] = errors
errors_df = errors_df.sort_values(by='Error', ascending=False)
errors_df[errors_df['Error'] > 0].describe(include='all')

In [ ]:
plt.figure(figsize=(20, 20))

# Group by 'Make' and calculate the mean error
mean_error_by_make = errors_df.groupby('Make')['Error'].mean().reset_index()

# Plot the mean error by make
plt.figure(figsize=(10, 15))
ax = sns.barplot(x='Error', y='Make', data=mean_error_by_make.sort_values(by='Error', ascending=False), orient='h')
sns.jointplot(x='Error', y=errors_df.groupby('Make').count(), hue='Make', data=mean_error_by_make)
ax.axvline(errors_df['Error'].mean(), color='red', linestyle='--')
plt.title('Средна абсолютна грешка по марка')
plt.ylabel('Марка')
plt.xlabel('Грешка')
plt.show()

# Eкспортиране на модела

In [ ]:
import pickle

with open('../models/random_forest_car_price_model.pkl', 'wb+') as model_file:
    pickle.dump(best_random_forest_reg_pipeline, model_file)